In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import os
import keras
import nltk
import re

/home/elk/anaconda3/envs/accidents/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
fp = 'Neiss_NLP_processed_1.csv'
df = pd.read_csv(fp, encoding='latin-1')  # so the original file is in the 'latin-1' encoding, so we must read decode it from this

/home/elk/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
low_memory=False

In [5]:
print(df.shape)

(1873078, 19)


In [6]:
print(list(df.columns.values))


['CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex', 'Race', 'Other_Race', 'Body_Part', 'Diagnosis', 'Other_Diagnosis', 'Disposition', 'Location', 'Fire_Involvement', 'Product_1', 'Product_2', 'Stratum', 'PSU', 'Weight', 'Narrative', 'Day_of_week']


In [8]:
df.dtypes

CPSC_Case_Number     object
Treatment_Date       object
Age                  object
Sex                   int64
Race                  int64
Other_Race           object
Body_Part             int64
Diagnosis             int64
Other_Diagnosis      object
Disposition           int64
Location              int64
Fire_Involvement      int64
Product_1             int64
Product_2             int64
Stratum              object
PSU                   int64
Weight              float64
Narrative            object
Day_of_week          object
dtype: object

In [28]:
narrative_series = df['Narrative']
print(type(narrative_series)) 

<class 'pandas.core.series.Series'>


In [38]:
pd.options.display.max_colwidth = 101
print(narrative_series)

0          54YOF, GOT UP AT HOME TO GO TO BATHOUROOM, FELL AND HIT HEAD ON BEDSIDE TABLE. Diagnosis is: LACE...
1          35YOF, SMOKE INHALATION  AND  SHOULDER PAIN FROM CARRYING BUCKETS OF WATERTO GRASS FIRE. Diagnosi...
2                  18YOM, HURT SHOULDER WHILE PLAYING BASKETBALL AT SCHOOL. Diagnosis is: R SHOULDERDISLOCATION
3                                     17YOM, STEPPED IN HOLE WHILE PLAYING FOOTBALL. Diagnosis is: R TOE SPRAIN
4                     49YOM, SLIPPED IN WATER ON FLOOR  IN JAIL. Diagnosis is: R ANKLE SPRAIN, R CHESTCONTUSION
5                       67YOF, CHAIR COLLASPED AFTER PATIENT SITTING DOWN IN IT. Diagnosis is: R KNEE CONTUSION
6                      15YOM, HEMATOMA TO ELBOW WHILE PLAYING FOOTBALL. Diagnosis is: INFECTED R ELBOW HEMATOMA
7                                         A 42YOM HAD BEEN DRINKING ETOH, FELL DOWN SEVERAL STAIRS, FX TO WRIST
8          A 69YOF CHANGED BODY WASH 2-3 WKS AGO(USES HIGHLY SCENTED SOAP NOW), DEVELOPED BODY RASH - BO

In [39]:
# Adjust the text to normalize for word separation for medical abrviasions

def medmore_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"HR", " hour ")
    df[text_field] = df[text_field].str.replace(r"[&]", " and ")
    df[text_field] = df[text_field].str.replace(r"DX", "[' '] diagnosis is ")
    df[text_field] = df[text_field].str.replace(r"C[/]O", " complains of ")
    df[text_field] = df[text_field].str.replace(r"@", " at ")
    df[text_field] = df[text_field].str.replace(r"WKS", " weeks ")
    df[text_field] = df[text_field].str.replace(r"FX", " fracture ")
    df[text_field] = df[text_field].str.replace(r"YRS", " years ")
    return df

df = medmore_text(df, "Narrative")

df.to_csv("Neiss_NLP_processed_2_medmore.csv", index=False)

df = pd.read_csv("Neiss_NLP_processed_2_medmore.csv")

df.head(30)



/home/elk/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Stratum,PSU,Weight,Narrative,Day_of_week
0,120103891,2012-01-02,54,2,1,NaN,75,59,NaN,1,1,0,4057,0,S,71,69.9872,"54YOF, GOT UP AT HOME TO GO TO BATHOUROOM, FELL AND HIT HEAD ON BEDSIDE TABLE. Diagnosis is: LACE...",Monday
1,120103905,2012-01-01,35,2,2,NaN,85,65,NaN,1,0,3,1143,0,S,71,69.9872,"35YOF, SMOKE INHALATION AND SHOULDER PAIN FROM CARRYING BUCKETS OF WATERTO GRASS FIRE. Diagnosi...",Sunday
2,120103908,2012-01-02,18,1,2,NaN,30,55,NaN,1,8,0,1205,0,S,71,69.9872,"18YOM, HURT SHOULDER WHILE PLAYING BASKETBALL AT SCHOOL. Diagnosis is: R SHOULDERDISLOCATION",Monday
3,120103909,2012-01-02,17,1,2,NaN,93,64,NaN,1,9,0,1211,0,S,71,69.9872,"17YOM, STEPPED IN HOLE WHILE PLAYING FOOTBALL. Diagnosis is: R TOE SPRAIN",Monday
4,120103911,2012-01-02,49,1,2,NaN,37,64,NaN,1,5,0,1807,0,S,71,69.9872,"49YOM, SLIPPED IN WATER ON FLOOR IN JAIL. Diagnosis is: R ANKLE SPRAIN, R CHESTCONTUSION",Monday
5,120103912,2012-01-02,67,2,2,NaN,35,53,NaN,1,1,0,4074,0,S,71,69.9872,"67YOF, CHAIR COLLASPED AFTER PATIENT SITTING DOWN IN IT. Diagnosis is: R KNEE CONTUSION",Monday
6,120103913,2012-01-02,15,1,2,NaN,32,58,NaN,1,9,0,1211,0,S,71,69.9872,"15YOM, HEMATOMA TO ELBOW WHILE PLAYING FOOTBALL. Diagnosis is: INFECTED R ELBOW HEMATOMA",Monday
7,120104529,2012-01-01,42,1,0,NaN,34,57,NaN,1,0,0,1842,0,V,25,16.2344,"A 42YOM HAD BEEN DRINKING ETOH, FELL DOWN SEVERAL STAIRS, fracture TO WRIST",Sunday
8,120104544,2012-01-01,69,2,2,NaN,85,74,NaN,1,0,0,983,0,V,25,16.2344,"A 69YOF CHANGED BODY WASH 2-3 WEEKS AGO(USES HIGHLY SCENTED SOAP NOW), DEVELOPED BODY RASH - BO...",Sunday
9,120104546,2012-01-01,23,2,2,NaN,75,71,HEADACHE,1,4,0,1141,0,V,25,16.2344,"A 23YOF IN LOW SPEED MVA WHEN CASE OF WATER FLEW FROM BACK SEAT AND HITPTS HEAD, Diagnosis is HEA...",Sunday


## Remove Symbols from "Narrative"

In [42]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"[\.,]", "")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

df = standardize_text(df, "Narrative")

df.to_csv("Neiss_NLP_processed_2_medmore_clean.csv", index=False)

df = pd.read_csv("Neiss_NLP_processed_2_medmore_clean.csv")

df.head(30)

/home/elk/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Stratum,PSU,Weight,Narrative,Day_of_week
0,120103891,2012-01-02,54,2,1,NaN,75,59,NaN,1,1,0,4057,0,S,71,69.9872,54yof got up at home to go to bathouroom fell and hit head on bedside table diagnosis is lacera...,Monday
1,120103905,2012-01-01,35,2,2,NaN,85,65,NaN,1,0,3,1143,0,S,71,69.9872,35yof smoke inhalation and shoulder pain from carrying buckets of waterto grass fire diagnosis...,Sunday
2,120103908,2012-01-02,18,1,2,NaN,30,55,NaN,1,8,0,1205,0,S,71,69.9872,18yom hurt shoulder while playing basketball at school diagnosis is r shoulderdislocation,Monday
3,120103909,2012-01-02,17,1,2,NaN,93,64,NaN,1,9,0,1211,0,S,71,69.9872,17yom stepped in hole while playing football diagnosis is r toe sprain,Monday
4,120103911,2012-01-02,49,1,2,NaN,37,64,NaN,1,5,0,1807,0,S,71,69.9872,49yom slipped in water on floor in jail diagnosis is r ankle sprain r chestcontusion,Monday
5,120103912,2012-01-02,67,2,2,NaN,35,53,NaN,1,1,0,4074,0,S,71,69.9872,67yof chair collasped after patient sitting down in it diagnosis is r knee contusion,Monday
6,120103913,2012-01-02,15,1,2,NaN,32,58,NaN,1,9,0,1211,0,S,71,69.9872,15yom hematoma to elbow while playing football diagnosis is infected r elbow hematoma,Monday
7,120104529,2012-01-01,42,1,0,NaN,34,57,NaN,1,0,0,1842,0,V,25,16.2344,a 42yom had been drinking etoh fell down several stairs fracture to wrist,Sunday
8,120104544,2012-01-01,69,2,2,NaN,85,74,NaN,1,0,0,983,0,V,25,16.2344,a 69yof changed body wash 2 3 weeks ago(uses highly scented soap now) developed body rash bod...,Sunday
9,120104546,2012-01-01,23,2,2,NaN,75,71,HEADACHE,1,4,0,1141,0,V,25,16.2344,a 23yof in low speed mva when case of water flew from back seat and hitpts head diagnosis is head...,Sunday
